In [7]:
import pandas as pd
import nltk
import numpy as np

from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

from collections import Counter

#fp = "../DATASETS/work_arrangements_development_set.csv"
fp = "../DATASETS/work_arrangements_test_set.csv"
df = pd.read_csv(fp)

[nltk_data] Downloading package punkt_tab to /home/jar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
def wa_classifier(text):
    """
    Baseline Bag-of-words frequency classifier
    """

    bow_freq = {
        "remote": 0,
        "onsite": 0,
        "hybrid": 0
    }
    text = word_tokenize(text.lower())

    token_count = Counter(text)
    for cat in bow_freq:
        bow_freq[cat] = token_count[cat]
    
    res = max(bow_freq, key=bow_freq.get)

    return (''.join(res[0].upper() + res[1:])) if res == "remote" or res == "hybrid" else "OnSite"  #capitalisation

#Apply
df['predicted_wa'] = df["job_ad"].apply(wa_classifier)


In [9]:
from eval import *
from pprint import pprint

#Accuracy
accuracy = np.mean(df['predicted_wa'] == df['y_true'])
print(f"\nAccuracy: {accuracy:.2%}")

# Print prediction vs ground truth
print("\nPrediction vs Ground Truth:\n")
for i, row in df.iterrows():
    predicted = row['predicted_wa']
    expected = row['y_true']
    if predicted != expected:
        print(f"[{i}] ❌ Predicted: {predicted} | Expected: {expected}")
    else:
        print(f"[{i}] ✅ Matched:   {predicted}")


p = precision(["Remote", "Hybrid", "OnSite"], df['y_true'], df['predicted_wa'])
pprint(p)


Accuracy: 42.42%

Prediction vs Ground Truth:

[0] ❌ Predicted: Remote | Expected: OnSite
[1] ❌ Predicted: Remote | Expected: OnSite
[2] ✅ Matched:   Remote
[3] ❌ Predicted: Remote | Expected: Hybrid
[4] ✅ Matched:   Remote
[5] ❌ Predicted: Remote | Expected: OnSite
[6] ❌ Predicted: Remote | Expected: Hybrid
[7] ✅ Matched:   Remote
[8] ❌ Predicted: Remote | Expected: Hybrid
[9] ✅ Matched:   Remote
[10] ✅ Matched:   Hybrid
[11] ❌ Predicted: Remote | Expected: OnSite
[12] ✅ Matched:   Remote
[13] ❌ Predicted: Remote | Expected: OnSite
[14] ❌ Predicted: Remote | Expected: OnSite
[15] ❌ Predicted: Remote | Expected: OnSite
[16] ❌ Predicted: Remote | Expected: OnSite
[17] ✅ Matched:   Remote
[18] ❌ Predicted: Remote | Expected: OnSite
[19] ✅ Matched:   Remote
[20] ✅ Matched:   Remote
[21] ✅ Matched:   Hybrid
[22] ✅ Matched:   Remote
[23] ✅ Matched:   Hybrid
[24] ❌ Predicted: Remote | Expected: OnSite
[25] ❌ Predicted: Remote | Expected: OnSite
[26] ✅ Matched:   Hybrid
[27] ❌ Predicted: Rem